In [24]:
import pandas as pd

county_names_file = "../original_data/all_counties_list.txt"

df_base = pd.read_csv(county_names_file, names=["COUNTY_MUNI_CODE"])
df_base.head()

,COUNTY_MUNI_CODE
0,MX01001
1,MX01002
2,MX01003
3,MX01004
4,MX01005


In [25]:
all_centroid_file = '../generated_files/centroids_us_mx.csv'
df_centroids = pd.read_csv(all_centroid_file)
df_centroids.head()

,COUNTY_MUNI_CODE,Latitude,Longitude
0,US31039,41.916641,-96.787401
1,US53069,46.291238,-123.433531
2,US35011,34.343066,-104.411894
3,US31109,40.784687,-96.687755
4,US31129,40.176603,-98.047185


In [26]:
row1 = ["MX04013", "20.3198", "-90.0568"]
row2 = ["MX07122", "16.3374", "-92.9744"]
comb_data = [row1, row2]
df_new = pd.DataFrame(comb_data, columns=["COUNTY_MUNI_CODE", "Latitude", "Longitude"])
df_centroids = pd.concat([df_centroids, df_new], ignore_index=True)
df_centroids.head()

,COUNTY_MUNI_CODE,Latitude,Longitude
0,US31039,41.916641,-96.787401
1,US53069,46.291238,-123.433531
2,US35011,34.343066,-104.411894
3,US31109,40.784687,-96.687755
4,US31129,40.176603,-98.047185


In [27]:
# counties not in centroid file
counties_base = set(df_base['COUNTY_MUNI_CODE'])
counties_centroids = set(df_centroids['COUNTY_MUNI_CODE'])
missing_counties = counties_base - counties_centroids
missing_counties

set()

In [28]:
df_base = df_centroids[df_centroids['COUNTY_MUNI_CODE'].isin(df_base['COUNTY_MUNI_CODE'])]

In [29]:
df_base

,COUNTY_MUNI_CODE,Latitude,Longitude
0,US31039,41.916641,-96.787401
2,US35011,34.343066,-104.411894
3,US31109,40.784687,-96.687755
4,US31129,40.176603,-98.047185
6,US46099,43.674431,-96.791436
...,...,...,...
5689,MX30203,20.13394,-97.558573
5690,MX15119,18.800148,-99.554097
5691,MX15120,19.809121,-99.071824
5692,MX04013,20.3198,-90.0568


### Finding nearest latitude and longitude according to .nc file for speed

In [30]:
df_base["nearest_latitude"] = None
df_base["nearest_longitude"] = None

/tmp/ipykernel_415113/488301065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base["nearest_latitude"] = None
/tmp/ipykernel_415113/488301065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base["nearest_longitude"] = None


In [32]:
import xarray as xr

path = "../ncs/r_250_mon_ERA5_1979-2025.nc"  # adjust if your file is elsewhere
ds = xr.open_dataset(path, decode_times=True)
ds

<xarray.Dataset> Size: 77MB
Dimensions:         (valid_time: 560, pressure_level: 1, latitude: 145,
                     longitude: 236)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 4kB 1979-01-01 ... 2025-08-01
  * pressure_level  (pressure_level) float64 8B 250.0
  * latitude        (latitude) float64 1kB 50.0 49.75 49.5 ... 14.5 14.25 14.0
  * longitude       (longitude) float64 2kB -125.2 -125.0 ... -66.75 -66.5
    expver          (valid_time) <U4 9kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 77MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-09-26T20:56 GRIB to CDM+CF via cfgrib-0.9.1...

In [33]:
all_lats = ds.latitude.values
all_longs = ds.longitude.values

In [34]:
import numpy as np
from scipy.spatial import cKDTree
from math import radians, cos

def build_kdtree(lat_grid, lon_grid):
    """
    Build a KDTree from 2D lat-lon grid (145x236).
    Returns:
        tree: cKDTree object
        points: array of shape (145*236, 2) containing all (lat, lon) pairs
    """
    # Create full coordinate mesh
    lat_mesh, lon_mesh = np.meshgrid(lat_grid, lon_grid, indexing='ij')
    points = np.column_stack((lat_mesh.ravel(), lon_mesh.ravel()))
    tree = cKDTree(points)
    return tree, points

In [35]:
tree, points = build_kdtree(all_lats, all_longs)

def find_nearest_point(lat_c, lon_c, tree, points):
    """
    Finds the nearest (lat, lon) from the KDTree to the county center.
    Returns: nearest_lat, nearest_lon, distance_deg
    """
    dist, idx = tree.query([lat_c, lon_c])
    nearest_lat, nearest_lon = points[idx]
    return nearest_lat, nearest_lon, dist

center_lat_sample = 22.9913638229323
center_lon_sample = -102.475691706378
nearest_lat, nearest_lon, dist = find_nearest_point(center_lat_sample, center_lon_sample, tree, points)
nearest_lat, nearest_lon, dist

(23.0, -102.5, 0.02579683494458176)

In [36]:
for i in range(len(df_base)):
    center_lat_sample = df_base.iloc[i]["Latitude"]
    center_lon_sample = df_base.iloc[i]["Longitude"]
    nearest_lat, nearest_lon, dist = find_nearest_point(center_lat_sample, center_lon_sample, tree, points)
    df_base.loc[df_base.index[i], "nearest_latitude"] = nearest_lat
    df_base.loc[df_base.index[i], "nearest_longitude"] = nearest_lon
df_base.head()

,COUNTY_MUNI_CODE,Latitude,Longitude,nearest_latitude,nearest_longitude
0,US31039,41.916641,-96.787401,42.0,-96.75
2,US35011,34.343066,-104.411894,34.25,-104.5
3,US31109,40.784687,-96.687755,40.75,-96.75
4,US31129,40.176603,-98.047185,40.25,-98.0
6,US46099,43.674431,-96.791436,43.75,-96.75


In [37]:
import numpy as np

num_unique_counties = len(df_base)
all_dates = pd.date_range(start="1979-01-01", end="2025-08-01", freq="MS")
print(len(all_dates))
df_monthly = pd.concat([df_base]*len(all_dates))
df_monthly = df_monthly.sort_index()
df_monthly.head()


560


,COUNTY_MUNI_CODE,Latitude,Longitude,nearest_latitude,nearest_longitude
0,US31039,41.916641,-96.787401,42.0,-96.75
0,US31039,41.916641,-96.787401,42.0,-96.75
0,US31039,41.916641,-96.787401,42.0,-96.75
0,US31039,41.916641,-96.787401,42.0,-96.75
0,US31039,41.916641,-96.787401,42.0,-96.75


In [38]:
df_monthly['date'] = np.tile(all_dates, num_unique_counties)
df_monthly

,COUNTY_MUNI_CODE,Latitude,Longitude,nearest_latitude,nearest_longitude,date
0,US31039,41.916641,-96.787401,42.0,-96.75,1979-01-01
0,US31039,41.916641,-96.787401,42.0,-96.75,1979-02-01
0,US31039,41.916641,-96.787401,42.0,-96.75,1979-03-01
0,US31039,41.916641,-96.787401,42.0,-96.75,1979-04-01
0,US31039,41.916641,-96.787401,42.0,-96.75,1979-05-01
...,...,...,...,...,...,...
5693,MX07122,16.3374,-92.9744,16.25,-93.0,2025-04-01
5693,MX07122,16.3374,-92.9744,16.25,-93.0,2025-05-01
5693,MX07122,16.3374,-92.9744,16.25,-93.0,2025-06-01
5693,MX07122,16.3374,-92.9744,16.25,-93.0,2025-07-01


In [39]:
col = 'date'
df_monthly = df_monthly[[col]+[c for c in df_monthly.columns if c!=col]]
df_monthly

,date,COUNTY_MUNI_CODE,Latitude,Longitude,nearest_latitude,nearest_longitude
0,1979-01-01,US31039,41.916641,-96.787401,42.0,-96.75
0,1979-02-01,US31039,41.916641,-96.787401,42.0,-96.75
0,1979-03-01,US31039,41.916641,-96.787401,42.0,-96.75
0,1979-04-01,US31039,41.916641,-96.787401,42.0,-96.75
0,1979-05-01,US31039,41.916641,-96.787401,42.0,-96.75
...,...,...,...,...,...,...
5693,2025-04-01,MX07122,16.3374,-92.9744,16.25,-93.0
5693,2025-05-01,MX07122,16.3374,-92.9744,16.25,-93.0
5693,2025-06-01,MX07122,16.3374,-92.9744,16.25,-93.0
5693,2025-07-01,MX07122,16.3374,-92.9744,16.25,-93.0


In [40]:
df_monthly.reset_index(drop=True, inplace=True)
df_monthly

,date,COUNTY_MUNI_CODE,Latitude,Longitude,nearest_latitude,nearest_longitude
0,1979-01-01,US31039,41.916641,-96.787401,42.0,-96.75
1,1979-02-01,US31039,41.916641,-96.787401,42.0,-96.75
2,1979-03-01,US31039,41.916641,-96.787401,42.0,-96.75
3,1979-04-01,US31039,41.916641,-96.787401,42.0,-96.75
4,1979-05-01,US31039,41.916641,-96.787401,42.0,-96.75
...,...,...,...,...,...,...
2014315,2025-04-01,MX07122,16.3374,-92.9744,16.25,-93.0
2014316,2025-05-01,MX07122,16.3374,-92.9744,16.25,-93.0
2014317,2025-06-01,MX07122,16.3374,-92.9744,16.25,-93.0
2014318,2025-07-01,MX07122,16.3374,-92.9744,16.25,-93.0


In [42]:
import pickle
import os

os.makedirs('../main_data', exist_ok=True)

with open('../main_data/base_file.pkl', 'wb') as f:
    pickle.dump(df_monthly, f)